In [1]:
import cv2
import numpy as np


In [2]:
# 自定义函数
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation=inter)
    return resized

def cv_show(img,name):
    cv2.imshow(name,img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [50]:
# 读取图片
img=cv2.imread('images/page.jpg')
img_copy=img.copy()
r=img.shape[1]/500
img=resize(img,width=500)
cv_show(img,'img')
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv_show(gray,'gray')
# 使用高斯滤波去模糊
gray=cv2.GaussianBlur(gray,(5, 5), 0)
cv_show(gray,'gray') # 效果是都变模糊了

In [8]:
# canny边缘检测
edge=cv2.Canny(gray,80,200)
cv_show(edge,'edge')

In [14]:
# 轮廓检测
cnts,_=cv2.findContours(edge.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(f'轮廓个数为：{len(cnts)}') #1个
# 寻找最大的那个轮廓
max_cnt=None
max_l=0
for cnt in cnts:
    l=cv2.arcLength(cnt,True)
    if l>max_l:
        max_l=l
        max_cnt=cnt
# 再将得到的最大轮廓进行近似，因为其边缘可能有一些尖刺
approx=cv2.approxPolyDP(max_cnt,0.02*max_l,True) # 四个坐标点
cv2.drawContours(img,[approx],-1,(0,0,255),2)
cv_show(img,'cnt')

轮廓个数为：1


In [30]:
"""
1.每一个cnt都有很多个坐标
2.approx只用4个坐标，连接起来就是一个四边形
"""
# 透视变换
"""
将原来的的近似轮廓四边形变为一个矩形并贴合边角
"""
def order_points(pts):
    # 一共4个坐标点
    rect = np.zeros((4, 2), dtype="float32")

    # 按顺序找到对应坐标0123分别是 左上，右上，右下，左下
    # 计算左上，右下
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # 计算右上和左下
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

def four_point_transform(image, pts):
    # 获取输入坐标点
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # 计算输入的w和h值
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # 变换后对应坐标位置
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    # 计算变换矩阵
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # 返回变换后结果
    return warped

r # 之前resize的比例
approx # 四个边角点
warped = four_point_transform(img_copy,approx.reshape(4,2)*r) # 透视变换

# 二值处理
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
ref = cv2.threshold(warped, 100, 255, cv2.THRESH_BINARY)[1]
cv_show(resize(ref,width=500),'ref')
cv2.imwrite('scan.jpg',ref)

True

In [43]:
# OCR文字识别
import pytesseract as ocr
import PIL

In [45]:
img=cv2.imread('scan.jpg',0)
# img=resize(img,width=500)
# thresh=cv2.threshold(img,200,255,cv2.THRESH_BINARY)[1]
cv_show(img,'img')

In [47]:
# text=ocr.image_to_string(PIL.Image.open('scan.jpg'))
text=ocr.image_to_string(img)

print(text)

JB ACCRSSING AND MANITLDALENG. PEXtES

On Line 14 we manipulate the top-left pixel in the im-
age, which is located at coordinate 11,0. and set it to have
avalue of 60, 0, 255). If we were reading this pirrel value
in RGB format, we would have a value of 9 for red, 9 for
green, and 255 tor blue, thus making, ita pure blue color

However, as | mentioned above, we need to take special
care when working, with OpenCY. Our pixels are actually
stored in BGR format, not RGB format

We actually read this pixel as 255 for red, 0 for green, and
0 for blue, making it a red color, not a blue color.

After setting the top-left pixel to have a red color on Line
14, we then grab the pixel value and print it back to con-
sole on Lines 15 and 16, just to demonstrate that we have
indeed successfully changed the color of the pixel

Accessing and setting a single pixel value is simple enough,
& o o ao
but what if we wanted to use NumPy’s array slicing capa-
bilities to access larger rectangular portions o

In [49]:
text=ocr.image_to_string(PIL.Image.open('scan.jpg'))
print(text)

Jd ACC RSSING AND MANITLDALENG PEXtES

On Line 14 we manipulate the top-left pixel in the im-
age, which is located at coordinate 1,0. and set it to have
avalue of 60, 0, 255). If we were reading this pirrel value
in RGB format, we would have a value of 9 for red, 9 for
green, and 255 tor blue, thus making ita pure blue color

However, as | mentioned above, we need to take special
care when working, with OpenCY. Our pixels are actually
stored in BGR format, not RGB format

We actually read this pixel as 255 for red, 0 for green, and
0 for blue, making it a red color, not a blue color.

After setting the top-left pixel to have a red color on Line
14, we then grab the pixel value and print it back to con-
sole on Lines 15 and 16, just to demonstrate that we have
indeed successfully changed the color of the pixel

Accessing and setting a single pixel value is simple enough,
& o o cr
but what if we wanted to use NumPy’s array slicing capa-
bilities to access larger rectangular portions of 